# 트리의 앙상블
- tree들이 여러개 묶여잇는 걸 숲이라고 한다 : 랜덤 포레스트
- 정형데이터에 대해 성능이 좋다
- 로또 추첨에서 나온 번호를 다시 기계 안에 넣어서 추첨하는 것처럼, 중복될 수도 있다는 특징있음

## 정형데이터
- 표, csv, 데이터베이스, 엑셀...

## 비정형데이터
- 쇼핑몰의 주문 리뷰, 댓글, 사진들, 디지털 음악

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://bit.ly/wine_csv_data')
data = df[['alcohol', 'sugar', 'pH']]
target = df[['class']]

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target)

In [8]:
# 레드와인인지 화이트와인인지를 분류하는 모델
from sklearn.model_selection import cross_validate # 데이터를 여러개 주기 위해서 불러옴
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1)

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) # rf 모델에 점수를 줄거야

In [10]:
print(scores) # 훈련세트에 과대적합 되어있긴 하다

{'fit_time': array([0.34221959, 0.36049151, 0.31162   , 0.23699784, 0.23893738]), 'score_time': array([0.02871609, 0.03071404, 0.0322113 , 0.03004837, 0.03188038]), 'test_score': array([0.89435897, 0.9025641 , 0.90041068, 0.88398357, 0.87371663]), 'train_score': array([0.99717732, 0.99769053, 0.99794767, 0.99846075, 0.99769112])}


In [13]:
rf.fit(train_input, train_target)
rf.feature_importances_ # 중요도를 숫자화 시킨 것

C:\Users\1-04\Desktop\DAMF2\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


array([0.2267278 , 0.50025744, 0.27301476])

- 각 수치는 알콜 당도 산도의 중요도이다
- == 당도의 중요도가 높다

- 중복이 되다보면 한 번도 선택되지 않은 데이터가 생기기 마련인데 이것을 OOB라고 한다

In [14]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1)
rf.fit(train_input, train_target)
rf.oob_score_

C:\Users\1-04\Desktop\DAMF2\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.895320197044335

## 엑스트라 트리
- 속도가 빠르다
- 노드를 나누는데 고민하는 시간이 줄고 일단 많이 테스트 해보는게 목적이라..
- 무작위로 특성을 뽑아서 테스트 한다. 성능이 조금 낮을 수는 있음

In [16]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1)

scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(scores)

{'fit_time': array([0.24813962, 0.24755144, 0.25635791, 0.18295169, 0.1830678 ]), 'score_time': array([0.03040814, 0.02979326, 0.02935433, 0.03009534, 0.02900958]), 'test_score': array([0.88615385, 0.90564103, 0.89425051, 0.89014374, 0.87166324]), 'train_score': array([0.99717732, 0.99769053, 0.99794767, 0.99846075, 0.99769112])}


## 그레디언트 부스팅
- 깊이가 얕은 결정트리를 많이 사용
- 스무고개에서 질문을 적게 하는거고 그래서 과대적합에 강하다
- 경사 하강법을 적용한 모델

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier() # 나무그루 기본 100그루 말고 500그루 하려면 `n_estimators=500` 추가

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(scores)

{'fit_time': array([0.215698  , 0.21357632, 0.33059621, 0.33632588, 0.33815002]), 'score_time': array([0.00355268, 0.00402761, 0.00520349, 0.0054338 , 0.00511551]), 'test_score': array([0.87897436, 0.87179487, 0.88193018, 0.86447639, 0.87166324]), 'train_score': array([0.89042853, 0.8894021 , 0.88532581, 0.89276552, 0.88917394])}


## 히스토그램 기반 그레이디언트 부스팅
- 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다
- 알콜 8.1~8.3까지 한 그룹으로 묶는 방식

In [20]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier() # 

scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(scores)

{'fit_time': array([0.12871623, 0.28470206, 0.27638388, 0.28162861, 0.27737927]), 'score_time': array([0.01327491, 0.01715875, 0.01923728, 0.01855493, 0.01775575]), 'test_score': array([0.88512821, 0.87589744, 0.89527721, 0.87268994, 0.86858316]), 'train_score': array([0.93122915, 0.93174237, 0.93714726, 0.93714726, 0.93996921])}


permutation_importance() 함수를 사용하면 특성값을 랜덤하게 섞는데 섞어서 점수가 비슷하면 중요하지 않은 특성이라고 정할 수 있다
- 어떤 특성이 가장 중요한가를 나타낸다. 수치가 높을수록 덜중요

In [21]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)

# scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(scores)

C:\Users\1-04\Desktop\DAMF2\ML\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'fit_time': array([0.12871623, 0.28470206, 0.27638388, 0.28162861, 0.27737927]), 'score_time': array([0.01327491, 0.01715875, 0.01923728, 0.01855493, 0.01775575]), 'test_score': array([0.88512821, 0.87589744, 0.89527721, 0.87268994, 0.86858316]), 'train_score': array([0.93122915, 0.93174237, 0.93714726, 0.93714726, 0.93996921])}
